In [1]:
import tensorflow as tf
import numpy as np

c:\users\windows\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = np.load('./helpers/data.npy')

In [3]:
my_dict = dict()
for d in data:
    if d.shape[0] not in my_dict:
        my_dict[d.shape[0]] = []
    my_dict[d.shape[0]].append(d)
    
for k in my_dict:
    my_dict[k] = np.stack(my_dict[k])
    print(my_dict[k].shape)

(3, 4, 4)
(2, 5, 4)
(7, 7, 4)
(43, 8, 4)
(35, 9, 4)
(18, 10, 4)
(228, 11, 4)
(506, 12, 4)
(628, 13, 4)
(431, 14, 4)
(1870, 15, 4)
(4256, 16, 4)
(7292, 17, 4)
(5418, 19, 4)
(37133, 20, 4)
(42408, 23, 4)


In [4]:
batch_size = 5
lstm_units = 50
vocab_size = 4
seq_len = 20

In [5]:
class CFGNetEncoder(tf.keras.layers.Layer):
    def __init__(self, lstm_units):
        super(CFGNetEncoder, self).__init__()
        self.lstm_units = lstm_units
        self.recurrent = tf.keras.layers.LSTM(lstm_units)
        self.dense1 = tf.keras.layers.Dense(lstm_units * 2)
        self.dense2 = tf.keras.layers.Dense(lstm_units * 2)
    
    def _sample(self, z_mean, z_log_sigma):
        batch_size = tf.shape(z_mean)[0]
        latent_dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch_size, latent_dim))
        return z_mean + tf.keras.backend.exp(z_log_sigma) * epsilon
    
    def call(self, v):
        out = self.recurrent(v)
        z_mean = self.dense1(out)
        z_log_sigma = self.dense2(out)
        return self._sample(z_mean, z_log_sigma)

# temp_encoder = CFGNetEncoder(lstm_units)
# y = tf.zeros((batch_size, lstm_units, vocab_size))
# state = temp_encoder(y)
# state.shape

In [6]:
class CFGNetDecoder(tf.keras.layers.Layer):
    def __init__(self, lstm_units, vocab_size):
        super(CFGNetDecoder, self).__init__()
        self.lstm_units = lstm_units
        self.vocab_size = vocab_size
        self.recurrent = tf.keras.layers.LSTM(lstm_units, return_sequences=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
 
    def call(self, seq_len, state):
        states = tf.split(state, num_or_size_splits=2, axis=1)
        hidden_state_shape = tf.shape(states[0])
        batch_size = hidden_state_shape[0]
        vocab_size = self.vocab_size
        v = tf.zeros((batch_size,seq_len,vocab_size))
        out = self.recurrent(v, initial_state=states)
        return self.dense(out)

# temp_decoder = CFGNetDecoder(lstm_units, vocab_size)
# state = tf.zeros((batch_size, lstm_units * 2))
# out = temp_decoder(seq_len, state)
# out.shape

In [7]:
class CFGNetAutoencoder(tf.keras.Model):
    def __init__(self, lstm_units,vocab_size):
        super(CFGNetAutoencoder, self).__init__()
        self.lstm_units = lstm_units
        self.vocab_size = vocab_size
        self.encoder = CFGNetEncoder(lstm_units)
        self.decoder = CFGNetDecoder(lstm_units, vocab_size)

    def call(self, v):
        seq_len = tf.shape(v)[1]
        state = self.encoder(v)
        return self.decoder(seq_len, state)

# temp_autoencoder = CFGNetAutoencoder(lstm_units, vocab_size)
# v = tf.zeros((batch_size, seq_len, vocab_size))
# out = temp_autoencoder(v)
# out.shape

In [8]:
data_to_use = my_dict[23]
data_to_use.shape

(42408, 23, 4)

In [9]:
cfg_net_autoencoder = CFGNetAutoencoder(100, data_to_use.shape[2])
cfg_net_autoencoder.compile(loss='categorical_crossentropy', optimizer='RMSprop')

In [10]:
# for key in my_dict:
for key in [23]:
    data_to_use = my_dict[key]
    tbCallback = tf.keras.callbacks.TensorBoard(log_dir='./logs/vae/RMSprop/%s'%(key), histogram_freq=0)
    cfg_net_autoencoder.fit(x=data_to_use,y=data_to_use,batch_size=10000,epochs=1000, callbacks=[tbCallback])

import winsound
frequency = 2500 
duration = 500 
winsound.Beep(frequency, duration)

Epoch 1/1000
4256/4256 [==============================] - 1s 338us/step - loss: 5.7847
Epoch 2/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.8434
Epoch 3/1000
4256/4256 [==============================] - 1s 219us/step - loss: 5.8133
Epoch 4/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.8179
Epoch 5/1000
4256/4256 [==============================] - 0s 117us/step - loss: 5.7689
Epoch 6/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7886
Epoch 7/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7819
Epoch 8/1000
4256/4256 [==============================] - 0s 110us/step - loss: 5.8034
Epoch 9/1000
4256/4256 [==============================] - 0s 117us/step - loss: 5.8000
Epoch 10/1000
4256/4256 [==============================] - 1s 120us/step - loss: 5.7804
Epoch 11/1000
4256/4256 [==============================] - 0s 111us/step - loss: 5.7650
Epoch 12/1000
4256/4256 [================

Epoch 93/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.7029
Epoch 94/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7632
Epoch 95/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.7843
Epoch 96/1000
4256/4256 [==============================] - 0s 114us/step - loss: 5.7819
Epoch 97/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.7573
Epoch 98/1000
4256/4256 [==============================] - 0s 117us/step - loss: 5.8792
Epoch 99/1000
4256/4256 [==============================] - 0s 114us/step - loss: 5.7553
Epoch 100/1000
4256/4256 [==============================] - 0s 108us/step - loss: 5.8513
Epoch 101/1000
4256/4256 [==============================] - 1s 136us/step - loss: 5.8062
Epoch 102/1000
4256/4256 [==============================] - 0s 117us/step - loss: 5.8269
Epoch 103/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7769
Epoch 104/1000
4256/4256 [==

Epoch 184/1000
4256/4256 [==============================] - 0s 117us/step - loss: 5.8169
Epoch 185/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.8446
Epoch 186/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.8514
Epoch 187/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.8233
Epoch 188/1000
4256/4256 [==============================] - 1s 125us/step - loss: 5.8659
Epoch 189/1000
4256/4256 [==============================] - 1s 119us/step - loss: 5.8525
Epoch 190/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.8373
Epoch 191/1000
4256/4256 [==============================] - 0s 111us/step - loss: 5.8212
Epoch 192/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7208
Epoch 193/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7479
Epoch 194/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7671
Epoch 195/1000
4256/4

4256/4256 [==============================] - 0s 108us/step - loss: 5.8137
Epoch 277/1000
4256/4256 [==============================] - 0s 110us/step - loss: 5.8440
Epoch 278/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.7700
Epoch 279/1000
4256/4256 [==============================] - 0s 115us/step - loss: 5.8163
Epoch 280/1000
4256/4256 [==============================] - 0s 114us/step - loss: 5.7875
Epoch 281/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7797
Epoch 282/1000
4256/4256 [==============================] - 0s 117us/step - loss: 5.8043
Epoch 283/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.8109
Epoch 284/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.8151
Epoch 285/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.8201
Epoch 286/1000
4256/4256 [==============================] - 0s 108us/step - loss: 5.8409
Epoch 287/1000
4256/4256 [==========

Epoch 367/1000
4256/4256 [==============================] - 0s 105us/step - loss: 5.8488
Epoch 368/1000
4256/4256 [==============================] - 0s 116us/step - loss: 5.7502
Epoch 369/1000
4256/4256 [==============================] - 1s 134us/step - loss: 5.7796
Epoch 370/1000
4256/4256 [==============================] - 1s 264us/step - loss: 5.8386
Epoch 371/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.8751
Epoch 372/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7927
Epoch 373/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7713
Epoch 374/1000
4256/4256 [==============================] - 0s 110us/step - loss: 5.8406
Epoch 375/1000
4256/4256 [==============================] - 1s 118us/step - loss: 5.8520
Epoch 376/1000
4256/4256 [==============================] - 0s 111us/step - loss: 5.8635
Epoch 377/1000
4256/4256 [==============================] - 0s 111us/step - loss: 5.7729
Epoch 378/1000
4256/4

Epoch 457/1000
4256/4256 [==============================] - 0s 108us/step - loss: 5.7654
Epoch 458/1000
4256/4256 [==============================] - 1s 129us/step - loss: 5.7899
Epoch 459/1000
4256/4256 [==============================] - 1s 121us/step - loss: 5.7607
Epoch 460/1000
4256/4256 [==============================] - 0s 117us/step - loss: 5.8135
Epoch 461/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.8037
Epoch 462/1000
4256/4256 [==============================] - 0s 105us/step - loss: 5.8166
Epoch 463/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.8475
Epoch 464/1000
4256/4256 [==============================] - 0s 117us/step - loss: 5.7516
Epoch 465/1000
4256/4256 [==============================] - 1s 121us/step - loss: 5.7770
Epoch 466/1000
4256/4256 [==============================] - 0s 113us/step - loss: 5.8889
Epoch 467/1000
4256/4256 [==============================] - 1s 132us/step - loss: 5.8606
Epoch 468/1000
4256/4

4256/4256 [==============================] - 1s 182us/step - loss: 5.7914
Epoch 550/1000
4256/4256 [==============================] - 0s 108us/step - loss: 5.8173
Epoch 551/1000
4256/4256 [==============================] - 1s 146us/step - loss: 5.8046
Epoch 552/1000
4256/4256 [==============================] - 0s 108us/step - loss: 5.7151
Epoch 553/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7803
Epoch 554/1000
4256/4256 [==============================] - 0s 108us/step - loss: 5.7542
Epoch 555/1000
4256/4256 [==============================] - 0s 106us/step - loss: 5.8105
Epoch 556/1000
4256/4256 [==============================] - 0s 117us/step - loss: 5.7498
Epoch 557/1000
4256/4256 [==============================] - 0s 107us/step - loss: 5.8364
Epoch 558/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.8271
Epoch 559/1000
4256/4256 [==============================] - 0s 106us/step - loss: 5.8555
Epoch 560/1000
4256/4256 [==========

Epoch 642/1000
4256/4256 [==============================] - 0s 109us/step - loss: 5.8135
Epoch 643/1000
4256/4256 [==============================] - 0s 112us/step - loss: 5.7946
Epoch 644/1000
4256/4256 [==============================] - 0s 117us/step - loss: 5.8195
Epoch 645/1000
4256/4256 [==============================] - 0s 107us/step - loss: 5.8033
Epoch 646/1000
4256/4256 [==============================] - 0s 107us/step - loss: 5.8023
Epoch 647/1000
4256/4256 [==============================] - 0s 113us/step - loss: 5.7381
Epoch 648/1000
4256/4256 [==============================] - 1s 122us/step - loss: 5.8248
Epoch 649/1000
4256/4256 [==============================] - 1s 119us/step - loss: 5.8116
Epoch 650/1000
4256/4256 [==============================] - 1s 118us/step - loss: 5.7950
Epoch 651/1000
4256/4256 [==============================] - 0s 111us/step - loss: 5.8487
Epoch 652/1000
4256/4256 [==============================] - 0s 114us/step - loss: 5.8235
Epoch 653/1000
4256/4

KeyboardInterrupt: 